In [1]:
import os
import sys

import logging
import re
import pathlib

import datetime

import numpy as np
import pandas as pd
from scipy.stats import pearsonr

import matplotlib.pyplot as plt

from pyplink import PyPlink
import seaborn as sns

import statsmodels.api as sm

import matplotlib.patches as patches
import matplotlib

from basic_tools import *

"""

jupyter nbconvert 5_association.ipynb --to script
python 5_association.py albumin 2 0

for i in {00..101};do python 5_association.py $i 2 1;done


for i in {00..10};do python 5_association.py $i 2 0;done
for i in {11..20};do python 5_association.py $i 2 0;done
for i in {21..30};do python 5_association.py $i 2 0;done
for i in {31..40};do python 5_association.py $i 2 0;done
for i in {41..50};do python 5_association.py $i 2 0;done
for i in {51..60};do python 5_association.py $i 2 0;done
for i in {61..70};do python 5_association.py $i 2 0;done
for i in {71..80};do python 5_association.py $i 2 0;done
for i in {91..90};do python 5_association.py $i 2 0;done
for i in {91..101};do python 5_association.py $i 2 0;done
for i in {51..60};do python 5_association.py $i 2 0;done
for i in {51..60};do python 5_association.py $i 2 0;done
for i in {51..60};do python 5_association.py $i 2 0;done

for i in {00..101};do python 5_association.py $i 1 0,2;done
for i in {00..101};do python 5_association.py $i 1 1;done

for i in {00..20};do python 5_association.py $i 1 1;done
for i in {21..40};do python 5_association.py $i 1 1;done
for i in {41..60};do python 5_association.py $i 1 1;done
for i in {61..80};do python 5_association.py $i 1 1;done
for i in {81..101};do python 5_association.py $i 1 1;done

"""

'\n\njupyter nbconvert 5_association.ipynb --to script\npython 5_association.py albumin 2 0\n\nfor i in {00..101};do python 5_association.py $i 2 1;done\n\n\nfor i in {00..10};do python 5_association.py $i 2 0;done\nfor i in {11..20};do python 5_association.py $i 2 0;done\nfor i in {21..30};do python 5_association.py $i 2 0;done\nfor i in {31..40};do python 5_association.py $i 2 0;done\nfor i in {41..50};do python 5_association.py $i 2 0;done\nfor i in {51..60};do python 5_association.py $i 2 0;done\nfor i in {61..70};do python 5_association.py $i 2 0;done\nfor i in {71..80};do python 5_association.py $i 2 0;done\nfor i in {91..90};do python 5_association.py $i 2 0;done\nfor i in {91..101};do python 5_association.py $i 2 0;done\nfor i in {51..60};do python 5_association.py $i 2 0;done\nfor i in {51..60};do python 5_association.py $i 2 0;done\nfor i in {51..60};do python 5_association.py $i 2 0;done\n\nfor i in {00..101};do python 5_association.py $i 1 0,2;done\nfor i in {00..101};do py

In [2]:
plink_KCHIP_HLA_AA_SNP_1000G=PyPlink(plink_KCHIP_HLA_AA_SNP_1000G_path)
plink_KCHIP_HLA_AA_SNP_1000G_fam=plink_KCHIP_HLA_AA_SNP_1000G.get_fam().astype({'fid':str,'iid':str}).rename(columns={'fid':'FID','iid':'IID'})
plink_KCHIP_HLA_AA_SNP_1000G_bim=plink_KCHIP_HLA_AA_SNP_1000G.get_bim()

In [3]:
phenotypes=pd.read_csv(pheno_all_file_path,sep='\t')
phenotypes=phenotypes.set_index('ID').loc[plink_KCHIP_HLA_AA_SNP_1000G_fam['IID']]

binary_continuous_traits=sorted(phenotypes.columns[~phenotypes.columns.str.contains('x_ray')])    

In [4]:
gene_bed_path='data/mart_export.txt'
gene_bed=pd.read_csv(gene_bed_path,sep='\t')
gene_bed=gene_bed.drop(columns='Exon stable ID')
gene_bed=gene_bed[(gene_bed['Gene start (bp)']>=plink_KCHIP_HLA_AA_SNP_1000G_bim.pos.min())&(gene_bed['Gene end (bp)']<=plink_KCHIP_HLA_AA_SNP_1000G_bim.pos.max())]
gene_bed=gene_bed[(gene_bed['Transcript type']=='protein_coding')]
gene_bed=gene_bed[~gene_bed.duplicated(['Gene name','Gene start (bp)','Gene end (bp)'])]
print(gene_bed.shape)
gene_bed=gene_bed[~gene_bed.duplicated(['Gene name'])]
print(gene_bed.shape)

gene_assign=plink_KCHIP_HLA_AA_SNP_1000G_bim[['pos']]

for idx,row in gene_bed.iterrows():
    gene_assign[row['Gene name']]=0
    
for idx,row in gene_bed.iterrows():    
    gene_assign[row['Gene name']][(gene_assign['pos']>=row['Gene start (bp)'])&(gene_assign['pos']<=row['Gene end (bp)'])]=1

#gene_assign.columns=gene_assign.columns.str.replace('HLA-','HLA_')        
    
HLA_names=np.unique([i[0].split('_')[1] for i in plink_KCHIP_HLA_AA_SNP_1000G_bim[plink_KCHIP_HLA_AA_SNP_1000G_bim.index.str.contains('HLA_')].index.str.split('*')])

for HLA_name in HLA_names:
    gene_select=gene_assign[gene_assign.index.str.contains('HLA_'+HLA_name)|gene_assign.index.str.contains('SNPS_'+HLA_name)|gene_assign.index.str.contains('AA_'+HLA_name)]#print(gene_select.sort_values('pos').iloc[0],gene_select.sort_values('pos').iloc[-1])
    HLA_name='HLA-{}'.format(HLA_name)
    gene_assign[HLA_name][(gene_assign['pos']>=gene_select['pos'].min())&(gene_assign['pos']<=gene_select['pos'].max())]=1 

(209, 30)
(207, 30)


/home/ch6845/tools/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [5]:
#gene_assign.iloc[:,1:].sum(axis=1).sort_values()

In [6]:
"""
gene_bed_path='data/known_genes_chr6.hg19.txt'
gene_bed=pd.read_csv(gene_bed_path,sep='\t')
gene_bed=gene_bed[(gene_bed['txStart']>=plink_KCHIP_HLA_AA_SNP_1000G_bim.pos.min())&(gene_bed['txEnd']<=plink_KCHIP_HLA_AA_SNP_1000G_bim.pos.max())]
gene_bed=gene_bed[~gene_bed.duplicated(['name2'])]

gene_assign=plink_KCHIP_HLA_AA_SNP_1000G_bim[['pos']]

for idx,row in gene_bed.iterrows():
    gene_assign[row['name2']]=0
    
for idx,row in gene_bed.iterrows():    
    gene_assign[row['name2']][(gene_assign['pos']>=row['txStart'])&(gene_assign['pos']<=row['txEnd'])]=1
gene_assign=gene_assign.drop(columns='HLA-A29.1')

gene_assign.columns=gene_assign.columns.str.replace('HLA-','HLA_')        
    
HLA_names=np.unique([i[0].split('_')[1] for i in plink_KCHIP_HLA_AA_SNP_1000G_bim[plink_KCHIP_HLA_AA_SNP_1000G_bim.index.str.contains('HLA_')].index.str.split('*')])

for HLA_name in HLA_names:
    gene_select=gene_assign[gene_assign.index.str.contains('HLA_'+HLA_name)|gene_assign.index.str.contains('SNPS_'+HLA_name)|gene_assign.index.str.contains('AA_'+HLA_name)]#print(gene_select.sort_values('pos').iloc[0],gene_select.sort_values('pos').iloc[-1])
    HLA_name='HLA_{}'.format(HLA_name)
    gene_assign[HLA_name][(gene_assign['pos']>=gene_select['pos'].min())&(gene_assign['pos']<=gene_select['pos'].max())]=1 
"""    

"\ngene_bed_path='data/known_genes_chr6.hg19.txt'\ngene_bed=pd.read_csv(gene_bed_path,sep='\t')\ngene_bed=gene_bed[(gene_bed['txStart']>=plink_KCHIP_HLA_AA_SNP_1000G_bim.pos.min())&(gene_bed['txEnd']<=plink_KCHIP_HLA_AA_SNP_1000G_bim.pos.max())]\ngene_bed=gene_bed[~gene_bed.duplicated(['name2'])]\n\ngene_assign=plink_KCHIP_HLA_AA_SNP_1000G_bim[['pos']]\n\nfor idx,row in gene_bed.iterrows():\n    gene_assign[row['name2']]=0\n    \nfor idx,row in gene_bed.iterrows():    \n    gene_assign[row['name2']][(gene_assign['pos']>=row['txStart'])&(gene_assign['pos']<=row['txEnd'])]=1\ngene_assign=gene_assign.drop(columns='HLA-A29.1')\n\ngene_assign.columns=gene_assign.columns.str.replace('HLA-','HLA_')        \n    \nHLA_names=np.unique([i[0].split('_')[1] for i in plink_KCHIP_HLA_AA_SNP_1000G_bim[plink_KCHIP_HLA_AA_SNP_1000G_bim.index.str.contains('HLA_')].index.str.split('*')])\n\nfor HLA_name in HLA_names:\n    gene_select=gene_assign[gene_assign.index.str.contains('HLA_'+HLA_name)|gene_assign

In [7]:
#gene_assign.columns[gene_assign.columns.str.contains('HLA')]

In [ ]:
gene_assign_reduce=gene_assign.iloc[:,:1]

gene_assign_reduce['gene_1']=gene_assign.iloc[:,1:].apply(lambda x: x[x!=0].index[1-1] if (x!=0).sum()>=(1) else np.nan,axis=1)
gene_assign_reduce['gene_2']=gene_assign.iloc[:,1:].apply(lambda x: x[x!=0].index[2-1] if (x!=0).sum()>=(2) else np.nan,axis=1)
gene_assign_reduce['gene_3']=gene_assign.iloc[:,1:].apply(lambda x: x[x!=0].index[3-1] if (x!=0).sum()>=(3) else np.nan,axis=1)
gene_assign_reduce['gene_4']=gene_assign.iloc[:,1:].apply(lambda x: x[x!=0].index[4-1] if (x!=0).sum()>=(4) else np.nan,axis=1)

In [55]:
#gene_assign_reduce.to_csv('data/bim_to_gene_assign.tsv',sep='\t')

In [5]:
#gene_assign_reduce
gene_assign_reduce=pd.read_csv('data/bim_to_gene_assign.tsv',sep='\t',index_col=0)

In [6]:
result_all=pd.read_csv('data/result_concat_1_top_all.tsv',sep='\t')
result_all['P_significant']=(result_all['P']<5e-8).astype(int)

In [7]:
len(result_all['marker_name']),len(result_all[result_all['P_significant']==1]['marker_name'].unique())

(163, 50)

In [14]:
#gene_assign_reduce

In [8]:
r2_df=pd.read_csv('data/result_concat_2_top_r2.tsv',sep='\t',index_col=0)

In [9]:
gene_assign.columns[gene_assign.columns.str.contains('HLA')]

Index(['HLA-F', 'HLA-G', 'HLA-A', 'HLA-E', 'HLA-C', 'HLA-B', 'HLA-DRA',
       'HLA-DRB5', 'HLA-DRB1', 'HLA-DQA1', 'HLA-DQB1', 'HLA-DQA2', 'HLA-DQB2',
       'HLA-DOB', 'HLA-DMB', 'HLA-DMA', 'HLA-DOA', 'HLA-DPA1', 'HLA-DPB1'],
      dtype='object')

In [10]:
'HLA-A', 'HLA-C', 'HLA-B'
'HLA-DRA','HLA-DRB1', 'HLA-DQA1', 'HLA-DQB1','HLA-DPA1', 'HLA-DPB1'

('HLA-DRA', 'HLA-DRB1', 'HLA-DQA1', 'HLA-DQB1', 'HLA-DPA1', 'HLA-DPB1')

In [11]:
def marker_name_to_genehow(marker_name):
    if marker_name[:3]=='AA_':
        ret=('HLA-'+marker_name.split('_')[1],'on_gene')
    elif marker_name[:5]=='SNPS_':
        ret=('HLA-'+marker_name.split('_')[1],'on_gene')
        #conditional_list.append('HLA_'+marker_name.split('_')[1])
    elif marker_name[:4]=='HLA_':
        ret=('HLA-'+marker_name.split('_')[1].split('*')[0],'on_gene')
    elif marker_name[:9]=='INS_SNPS_':
        ret=('HLA-'+marker_name.split('_')[2],'on_gene')
    elif np.any(~gene_assign_reduce.loc[marker_name].iloc[1:].isnull()):
        ret=gene_assign_reduce.loc[marker_name].iloc[1:]
        ret=(','.join(ret[~ret.isnull()]),'on_gene')
    else:
        r2=r2_df[marker_name].sort_values(ascending=False)
        gene_assign_r2_moderate_HLA=gene_assign.loc[r2[r2>0.5].index][gene_assign.columns[gene_assign.columns.str.contains('HLA')].tolist()+['MICA','MICB','TAP1','TAP2']]
        gene_assign_r2_moderate_HLA=gene_assign_r2_moderate_HLA[gene_assign_r2_moderate_HLA.sum(axis=1)!=0]
        #print(gene_assign_r2_moderate_HLA.sum(axis=1)!=0)
        
        gene_assign_r2_strong=gene_assign.loc[r2[r2>0.8].index].iloc[:,1:]
        gene_assign_r2_strong=gene_assign_r2_strong[gene_assign_r2_strong.sum(axis=1)!=0]
        #gene_assign_r2_strong_sum=gene_assign_r2_strong.sum()
        #
        if len(gene_assign_r2_moderate_HLA)!=0:
            ret=gene_assign_r2_moderate_HLA.iloc[0].sort_values(ascending=False).index[0]
            ret=(ret,'moderate_LD')
            print(marker_name)
            print(gene_assign_r2_moderate_HLA.iloc[:10])
        elif len(gene_assign_r2_strong)!=0:
            ret=gene_assign_r2_strong.iloc[0].sort_values(ascending=False).index[0]#[gene_assign_r2_strong.iloc[0]==1]
            ret=(ret,'strong_LD')
        else:
            #print(marker_name)
            gene_assign_temp=gene_assign.copy()
            gene_assign_temp['dist']=(gene_assign_temp['pos']-gene_assign_temp['pos'][marker_name]).abs()
            #print(marker_name,gene_assign_temp.sort_values('dist'))
            
            gene_assign_temp_clip=gene_assign_temp.sort_values('dist').iloc[:,1:-1]
            
            for row_idx,row in gene_assign_temp_clip.iterrows():
                if np.any(row!=0):
                    #print(row)
                    ret=','.join(row[row!=0].index)
                    ret=(ret,'nearest')
                    break
    return ret


"""
def marker_name_to_gene(marker_name):
    if marker_name[:3]=='AA_':
        ret='HLA_'+marker_name.split('_')[1]
    elif marker_name[:5]=='SNPS_':
        ret='HLA_'+marker_name.split('_')[1]
        #conditional_list.append('HLA_'+marker_name.split('_')[1])
    elif marker_name[:4]=='HLA_':
        ret='HLA_'+marker_name.split('_')[1].split('*')[0]
    elif marker_name[:9]=='INS_SNPS_':
        ret='HLA_'+marker_name.split('_')[2]
    elif np.any(~gene_assign_reduce.loc[marker_name].iloc[1:].isnull()):
        ret=gene_assign_reduce.loc[marker_name].iloc[1:]
        ret=','.join(ret[~ret.isnull()])
    else:
        r2=r2_df[marker_name].sort_values(ascending=False)
        gene_assign_r2_moderate_HLA=gene_assign.loc[r2[r2>0.5].index][gene_assign.columns[gene_assign.columns.str.contains('HLA')]]
        gene_assign_r2_moderate_HLA=gene_assign_r2_moderate_HLA[gene_assign_r2_moderate_HLA.sum(axis=1)!=0]
        #print(gene_assign_r2_moderate_HLA.sum(axis=1)!=0)
        
        gene_assign_r2_strong=gene_assign.loc[r2[r2>0.8].index].iloc[:,1:]
        gene_assign_r2_strong=gene_assign_r2_strong[gene_assign_r2_strong.sum(axis=1)!=0]
        #gene_assign_r2_strong_sum=gene_assign_r2_strong.sum()
        #
        if len(gene_assign_r2_moderate_HLA)!=0:
            ret=gene_assign_r2_moderate_HLA.iloc[0].sort_values(ascending=False).index[0]
        elif len(gene_assign_r2_strong)!=0:
            ret=gene_assign_r2_strong.iloc[0].sort_values(ascending=False).index[0]#[gene_assign_r2_strong.iloc[0]==1]
        else:
            #print(marker_name)
            gene_assign_temp=gene_assign.copy()
            gene_assign_temp['dist']=(gene_assign_temp['pos']-gene_assign_temp['pos'][marker_name]).abs()
            #print(marker_name,gene_assign_temp.sort_values('dist'))
            
            gene_assign_temp_clip=gene_assign_temp.sort_values('dist').iloc[:,1:-1]
            
            for row_idx,row in gene_assign_temp_clip.iterrows():
                if np.any(row!=0):
                    #print(row)
                    ret=','.join(row[row!=0].index)
                    break
    return ret
"""

"\ndef marker_name_to_gene(marker_name):\n    if marker_name[:3]=='AA_':\n        ret='HLA_'+marker_name.split('_')[1]\n    elif marker_name[:5]=='SNPS_':\n        ret='HLA_'+marker_name.split('_')[1]\n        #conditional_list.append('HLA_'+marker_name.split('_')[1])\n    elif marker_name[:4]=='HLA_':\n        ret='HLA_'+marker_name.split('_')[1].split('*')[0]\n    elif marker_name[:9]=='INS_SNPS_':\n        ret='HLA_'+marker_name.split('_')[2]\n    elif np.any(~gene_assign_reduce.loc[marker_name].iloc[1:].isnull()):\n        ret=gene_assign_reduce.loc[marker_name].iloc[1:]\n        ret=','.join(ret[~ret.isnull()])\n    else:\n        r2=r2_df[marker_name].sort_values(ascending=False)\n        gene_assign_r2_moderate_HLA=gene_assign.loc[r2[r2>0.5].index][gene_assign.columns[gene_assign.columns.str.contains('HLA')]]\n        gene_assign_r2_moderate_HLA=gene_assign_r2_moderate_HLA[gene_assign_r2_moderate_HLA.sum(axis=1)!=0]\n        #print(gene_assign_r2_moderate_HLA.sum(axis=1)!=0)\n  

In [12]:
result_all[result_all['phenotype_name']=='hdl_cholesterol']
result_all[result_all['phenotype_name']=='bun']

,Unnamed: 0,Unnamed: 0.1,phenotype_name,samples(case/control),step,condition,marker_name,note,term,POS,...,A1_freq_control,multi_allele,nobs,Z,coef,std,chisq,df,P,P_significant
32,18880,169281,bun,124563,1,NaN,6:30737591_G/C,unphased bialleic,ADD,30737591,...,NaN,NaN,124563,-6.93374,-0.186481,0.026895,NaN,NaN,4.118390e-12,1
33,13019,129480,bun,124563,2,6:30737591_G/C,6:29922341_G/C,unphased bialleic,ADD,29922341,...,NaN,NaN,124563,4.52162,0.068095,0.015060,NaN,NaN,6.142420e-06,0


In [13]:
#r2_df.loc['6:29896680_A/G']

In [14]:
result_all['gene_genehow']=np.nan
result_all['gene']=np.nan
result_all['genehow']=np.nan

result_all['gene_genehow'][result_all['P_significant']==1]=\
result_all[result_all['P_significant']==1].apply(lambda x: marker_name_to_genehow(x['marker_name']),axis=1)

result_all['gene'][result_all['P_significant']==1]=result_all[result_all['P_significant']==1].apply(lambda x: x['gene_genehow'][0],axis=1)
result_all['genehow'][result_all['P_significant']==1]=result_all[result_all['P_significant']==1].apply(lambda x: x['gene_genehow'][1],axis=1)

6:31351530_T/C
                HLA-F  HLA-G  HLA-A  HLA-E  HLA-C  HLA-B  HLA-DRA  HLA-DRB5  \
snp                                                                           
6:31376545_C/T      0      0      0      0      0      0        0         0   

                HLA-DRB1  HLA-DQA1  ...  HLA-DOB  HLA-DMB  HLA-DMA  HLA-DOA  \
snp                                 ...                                       
6:31376545_C/T         0         0  ...        0        0        0        0   

                HLA-DPA1  HLA-DPB1  MICA  MICB  TAP1  TAP2  
snp                                                         
6:31376545_C/T         0         0     1     0     0     0  

[1 rows x 23 columns]
6:29896680_A/G
                              HLA-F  HLA-G  HLA-A  HLA-E  HLA-C  HLA-B  \
snp                                                                      
AA_A_144_29911204_exon3           0      0      1      0      0      0   
SNPS_A_873_29911203_exon3         0      0      1      0      0   

/home/ch6845/tools/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/ch6845/tools/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/ch6845/tools/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [30]:
result_all[result_all['genehow']=='moderate_LD'].head()

/home/ch6845/tools/miniconda3/lib/python3.7/site-packages/pandas/core/ops.py:1649: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


,Unnamed: 0,Unnamed: 0.1,phenotype_name,samples(case/control),step,condition,marker_name,note,term,POS,...,Z,coef,std,chisq,df,P,P_significant,gene_genehow,gene,genehow


In [16]:
result_all

,Unnamed: 0,Unnamed: 0.1,phenotype_name,samples(case/control),step,condition,marker_name,note,term,POS,...,Z,coef,std,chisq,df,P,P_significant,gene_genehow,gene,genehow
0,20730,144613,FEV_over_FVC_predicted,7440,1,NaN,6:30985177_G/A,unphased bialleic,ADD,30985177,...,-5.183770,-0.060131,0.011600,NaN,NaN,2.231720e-07,0,NaN,NaN,NaN
1,68111,475104,FEV_predicted,7440,1,NaN,6:34172055_C/A,unphased bialleic,ADD,34172055,...,5.836440,0.026954,0.004618,NaN,NaN,5.556790e-09,1,"(HMGA1, strong_LD)",HMGA1,strong_LD
2,21231,169288,FEV_predicted,7440,2,6:34172055_C/A,6:31018788_T/G,unphased bialleic,ADD,31018788,...,-4.352160,-0.015100,0.003470,NaN,NaN,1.366070e-05,0,NaN,NaN,NaN
3,68167,475496,FVC_predicted,7441,1,NaN,6:34205465_G/GGAGCCC,unphased bialleic,ADD,34205465,...,5.573560,0.036813,0.006605,NaN,NaN,2.583340e-08,1,"(HMGA1, on_gene)",HMGA1,on_gene
4,21221,169200,FVC_predicted,7441,2,6:34205465_G/GGAGCCC,AX-11396301,unphased bialleic,ADD,31017334,...,4.630100,0.022921,0.004950,NaN,NaN,3.717070e-06,0,NaN,NaN,NaN
5,29301,204407,MVV_predicted,7441,1,NaN,SNPS_B_293_31324643_exon2,unphased bialleic,ADD,31324643,...,-3.290540,-0.415866,0.126382,NaN,NaN,1.004640e-03,0,NaN,NaN,NaN
6,55898,445472,acute_liver_disease,669/75509,1,NaN,6:32658375_T/G,unphased bialleic,ADD,32658375,...,3.990650,NaN,NaN,NaN,NaN,6.589200e-05,0,NaN,NaN,NaN
7,12169,154,albumin,125069,1,NaN,HLA_C,phased multialleic,NaN,29910247,...,NaN,NaN,NaN,132.950396,15.0,5.614026e-21,1,"(HLA-C, on_gene)",HLA-C,on_gene
8,68169,1630320,albumin,125069,2,HLA_C,6:34205822_T/C,unphased bialleic,ADD,34205822,...,-8.341820,-0.012715,0.001524,NaN,NaN,7.389020e-17,1,"(HMGA1, on_gene)",HMGA1,on_gene
9,17528,436425,albumin,125069,3,"6:34205822_T/C,HLA_C",6:30426103_A/T,unphased bialleic,ADD,30426103,...,-4.833360,-0.005750,0.001190,NaN,NaN,1.344090e-06,0,NaN,NaN,NaN


In [17]:
result_all[result_all['P_significant']==1]

,Unnamed: 0,Unnamed: 0.1,phenotype_name,samples(case/control),step,condition,marker_name,note,term,POS,...,Z,coef,std,chisq,df,P,P_significant,gene_genehow,gene,genehow
1,68111,475104,FEV_predicted,7440,1,NaN,6:34172055_C/A,unphased bialleic,ADD,34172055,...,5.83644,0.026954,0.004618,NaN,NaN,5.556790e-09,1,"(HMGA1, strong_LD)",HMGA1,strong_LD
3,68167,475496,FVC_predicted,7441,1,NaN,6:34205465_G/GGAGCCC,unphased bialleic,ADD,34205465,...,5.57356,0.036813,0.006605,NaN,NaN,2.583340e-08,1,"(HMGA1, on_gene)",HMGA1,on_gene
7,12169,154,albumin,125069,1,NaN,HLA_C,phased multialleic,NaN,29910247,...,NaN,NaN,NaN,132.950396,15.0,5.614026e-21,1,"(HLA-C, on_gene)",HLA-C,on_gene
8,68169,1630320,albumin,125069,2,HLA_C,6:34205822_T/C,unphased bialleic,ADD,34205822,...,-8.34182,-0.012715,0.001524,NaN,NaN,7.389020e-17,1,"(HMGA1, on_gene)",HMGA1,on_gene
10,53742,481860,allergic_disease,9273/80451,1,NaN,6:32632237_C/T,unphased bialleic,ADD,32632237,...,5.65399,NaN,NaN,NaN,NaN,1.567660e-08,1,"(HLA-DQB1, on_gene)",HLA-DQB1,on_gene
12,30408,272754,alt,120788,1,NaN,6:31351530_T/C,unphased bialleic,ADD,31351530,...,6.52296,0.326987,0.050129,NaN,NaN,6.920490e-11,1,"(MICA, moderate_LD)",MICA,moderate_LD
13,68119,678800,alt,120788,2,6:31351530_T/C,6:34178764_A/C,unphased bialleic,ADD,34178764,...,-5.47584,-0.334937,0.061166,NaN,NaN,4.362960e-08,1,"(HMGA1, strong_LD)",HMGA1,strong_LD
17,41462,217,ast,121162,1,NaN,HLA_DRB1,phased multialleic,NaN,32546547,...,NaN,NaN,NaN,142.453002,20.0,1.731522e-20,1,"(HLA-DRB1, on_gene)",HLA-DRB1,on_gene
18,62234,1797971,ast,121162,2,HLA_DRB1,SNPS_DPB1_8680_33052498_intron2_A,unphased bialleic,ADD,33052498,...,-6.15249,-0.210949,0.034287,NaN,NaN,7.651550e-10,1,"(HLA-DPB1, on_gene)",HLA-DPB1,on_gene
24,11573,92584,blood_in_urine,1817/15280,1,NaN,6:29896680_A/G,unphased bialleic,ADD,29896680,...,-5.54189,NaN,NaN,NaN,NaN,2.992310e-08,1,"(HLA-A, moderate_LD)",HLA-A,moderate_LD


In [97]:
result_all.to_csv('data/result_concat_3_top_gene_annot.tsv',sep='\t',index=None)

In [100]:
#result_all[result_all['phenotype_name']=='glucose_in_blood']

In [286]:
for i in result_all['gene'][result_all['step']!=1]:
    print(i)

HCG22
HCG22
HMGA1
HLA_E
HLA_A
HMGA1
HLA_DQA1
HLA_DPB1
HMGA1
HLA_A
TRIM27
C6orf1
HLA_A
NOTCH4
HLA_DRB1
HMGA1
NOTCH4
ITPR3
HLA_B
HMGA1
UHRF1BP1
HCG22
HLA_DPB1
HLA_C
SCAND3
TAF11
IER3
LSM2
HLA_B
LOC285830
FLJ43752
HCG9
HLA_DRB1
TRIM27
RPS10
HCG26
HMGA1
GRM4
GRM4
GRM4
ZSCAN16
HLA_B
BAK1
HLA_DRB1
HMGA1
HLA_A
HLA_DPB1
ZFP57
HMGA1
ATF6B
HLA_A
HLA_B
HLA_DRB1
HLA_B
HLA_C
HLA_DOA
HMGA1
SKIV2L
HLA_DRB1
HLA_B
HMGA1
HLA_A
HLA_DQA1
TRIM27
HCG22
HLA_A
